# Alright, now we have to consolidate our data. 

### We have 3 major sources. The United Nations, the World Bank, and of course, our inequality data. We also have 2 sources which have the same format - we just treat this as its own source called Misc. data. Within each source the data and naming conventions are standardized, however, the conventions are different across sources. However we want to be able to join all of them together. Let's make an entity for each of the sources first, then we can worry about merging

### Let's take the inequality index first and use that as our baseline. Because of naming inconsistencies, we are building a dictionary that gives us various country to code translations. Then, we introduce the 4 sources one at a time, sort out the countries which do not have a match, then go to the next one till as many countries as we can have corresponding codes

In [1]:
import pandas as pd
import numpy as np
data = pd.DataFrame
path = "DataSources\\swiiidGINI.csv"
data = pd.read_csv(path)

In [2]:
codes = pd.DataFrame
path = "DataSources\\codeTrans.csv"
codes = pd.read_csv(path)
codes.columns
codes["country"] = codes["name"]
codes = codes[["country", "alpha-3"]]

In [3]:
temp = codes.merge(data, on = "country", how = "right")
miss = (set(temp[temp["alpha-3"].isna()]["country"]))
miss = ['Bolivia',
 'Brunei',
 'Cape Verde',
 'Congo-Brazzaville',
 'Congo-Kinshasa',
 'Czech Republic',
 'Czechoslovakia',
 'Iran',
 'Korea',
 'Kosovo',
 'Laos',
 'Macedonia',
 'Micronesia',
 'Moldova',
 'Palestinian Territories',
 'Russia',
 'Soviet Union',
 'St. Kitts and Nevis',
 'St. Lucia',
 'St. Vincent and Grenadines',
 'Swaziland',
 'Syria',
 'São Tomé and Príncipe',
 'Taiwan',
 'Tanzania',
 'United Kingdom',
 'United States',
 'Venezuela',
 'Vietnam',
 'Yugoslavia']
#Source https://laendercode.net/en/3-letter-list.html
# note that korea is SOuth Korea
codes = ["BOL", "BRN", "CPV", "COG", "COD", "CZE", "NO1", "IRN", "KOR", "XKX", "LAO", "MKD", "FSM", "MDA", "PSE", "RUS", 
        "NO2", "KNA", "LCA", "VCT", "SWZ", "SYR", "STP", "TWN", "TZA", "GBR", "USA", "VEN", "VNM", "YUG"]
tc = {}
for cnt in range(len(codes)):
    #print(miss[cnt])
    tc[miss[cnt]] = codes[cnt]
for key, val in tc.items():
    temp.loc[temp["country"] == key, "alpha-3"] = val

In [4]:
d1 = {y:x for x,y in zip(temp["country"],temp["alpha-3"])}

In [5]:
codes = pd.read_csv(path)
codes.columns
codes["country"] = codes["name"]
codes = codes[["country", "alpha-3"]]
d2 = {y:x for x,y in zip(codes["country"],codes["alpha-3"])}

In [6]:
from collections import defaultdict
dd = defaultdict(list)

for d in (d1, d2): # you can list as many input dicts as you want here
    for key, value in d.items():
        if value not in dd[key]:
            dd[key].append(value)

In [7]:
path = "DataSources\\trade.csv"
data = pd.read_csv(path, header = 4)
data.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [8]:
hcnts = set(data["Country Name"])

## Manually added to our dictionary as data was added. Unadded countries were checked with something like:

#### set(data3[data3["code"].isna()]["country"])

## After applying the mapping in our dictionary. Then we decided whether to keep it or not (for instance, it is pointless to keep west gemany since out time period focus is after it exen existed)

In [9]:
rdd = {}
for key,val in dd.items():
    for x in val:
        rdd[x] = key

rdd['Yemen North'] = "YEM"
rdd['Myanmar (Burma)'] = "MMR"
rdd["Moldova (Republic of)"] = "MDA"
rdd["Korea (Democratic People's Rep. of)"] = "PRK"
rdd["Korea (Republic of)"] = "KOR"
rdd["The former Yugoslav Republic of Macedonia"] = "MKD"
rdd["Hong Kong, China (SAR)"] = "HKG"
rdd["Tanzania (United Republic of)"] = "TZA"
rdd["Eswatini (Kingdom of)"] = "SWZ"
rdd["Congo (Democratic Republic of the)"] = "COD"
rdd["Korea North"] = "PRK"
rdd["Slovak Republic"] = "SVK"
rdd["Dominican Rep"] = "DOM"
rdd["Bosnia"] = "BOS"
rdd["Serbia and Montenegro"] = "YUG"
rdd["Yugoslavia"] = "YUG"
rdd["Congo Brazzaville"] = "COG"
rdd["UAE"] = "UAE"
rdd["Ivory Coast"] = "CIV"
rdd["Korea South"] = "KOR"
rdd["Egypt, Arab Rep."] = "EGY"
rdd["Congo Kinshasa"] = "COD"
rdd['East Timor'] = "TLS"

## Here 2 dataframes will act as bases for our tables in sql. country_code and country_name

In [10]:
country_code = pd.DataFrame(list(rdd.items()), columns = ["cntr_name", "code"])

In [11]:
codes_table = pd.DataFrame(list(set(rdd.values())), columns = ["code"])

## Let's make the CSVs and put it in SQL

## Start by making baseline inequality

### That is, we need to fill in missing indices. That is for instance, if we do not have data for Bolivia in 2008, we still need an empty row representing Bolivia 2008. 

In [12]:
import pandas as pd
import numpy as np
data = pd.DataFrame
path = "DataSources\\swiiidGINI.csv"
data = pd.read_csv(path)

codes = pd.DataFrame
path = "DataSources\\codeTrans.csv"
codes = pd.read_csv(path)

codes.columns
codes["country"] = codes["name"]
codes = codes[["country", "alpha-3"]]

data["code"] = data["country"].map(rdd)
temp = data

In [13]:
cntrConv = list(set(list(zip(temp["country"], temp["code"]))))

masterDict = {}
masterDict["country"] = []
masterDict["code"] = []
masterDict["year"] = []
masterDict["gini_disp"] = []
masterDict["gini_disp_se"] = []
masterDict["gini_mkt"] = []
masterDict["gini_mkt_se"] = []
for x in cntrConv:
    for year in range(1996, 2017):
        masterDict["country"].append(x[0])
        masterDict["code"].append(x[1])
        masterDict["year"].append(year)
        try:
            masterDict["gini_disp"].append(temp[temp["code"] == x[1]][temp["year"] == year]["gini_disp"].values[0])
        except IndexError:
            masterDict["gini_disp"].append(np.nan)
        try:
            masterDict["gini_disp_se"].append(temp[temp["code"] == x[1]][temp["year"] == year]["gini_disp_se"].values[0])
        except IndexError:
            masterDict["gini_disp_se"].append(np.nan)
        try:
            masterDict["gini_mkt"].append(temp[temp["code"] == x[1]][temp["year"] == year]["gini_mkt"].values[0])
        except IndexError:
            masterDict["gini_mkt"].append(np.nan)
        try:
            masterDict["gini_mkt_se"].append(temp[temp["code"] == x[1]][temp["year"] == year]["gini_mkt_se"].values[0])
        except IndexError:
            masterDict["gini_mkt_se"].append(np.nan)

C:\Users\Viral Shanker\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Viral Shanker\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Viral Shanker\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Viral Shanker\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [14]:
master =  pd.DataFrame.from_dict(masterDict)

## Now let's get the other data from the world bank and the UN. The trick is, the data is given differently. There is Country - indicator, and then the columns are years. We must transpose and seperate! 

### First let's make some utility functions to make adding painless. We will use our dictionary rdd do convert country names into codes for the UN. For the World bank, we're lucky and the code is included in the dataset itself.

In [15]:

## The ren flag is activated for specific World Bank data which names years slightly differently
def addWorldBankVar(data, varName, ren = False):
    data["code"] = data["Country Code"]
    if ren:
        data.columns = ['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1996', '1998', '2000', '2002',
       '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', "code"]
        yrs = ['1996', '1998', '2000', '2002',
       '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018']
        data = data[["code"] + yrs]
        data = data.replace("..", np.nan)
    else:
        data = data[["code"] + [str(x) for x in list(range(1996,2017))]]
    data = pd.melt(data, id_vars = ["code"], var_name = "year", value_name = varName)
    data = data.sort_values(["code", "year"])
    data["year"] = np.int_(data["year"])
    data = data[data["code"].isin(master["code"])]
    data = data.set_index(['code','year'], drop = True)
    return data

def addUNData(data, varName):
    data = data.replace("..", np.nan)
    data["code"] = data["Country"].map(rdd)
    data = data[["code"] + [str(x) for x in list(range(1996,2017))]]
    data = pd.melt(data, id_vars = ["code"], var_name = "year", value_name = varName)
    data = data.sort_values(["code", "year"])
    data["year"] = np.int_(data["year"])
    data = data[data["code"].isin(master["code"])]
    data = data.set_index(['code','year'], drop = True)
    return data
    
def addNormalData(data, varName):
    data = data.sort_values(["code", "year"])
    data["year"] = np.int_(data["year"])
    data = data[data["year"].isin(list(range(1996,2017)))]
    data = data[data["code"].isin(master["code"])]
    data = data.set_index(['code','year'], drop = True)
    return data

## The 4 UN variables

In [16]:
#annoyingly, these are in LATIN-1 instead of UTF-8, hence we need the argument

path = "DataSources\\hdi.csv"
data1 = pd.read_csv(path, encoding = "LATIN-1", header = 1)

path = "DataSources\\EDU_IDX.csv"
data2 = pd.read_csv(path, encoding = "LATIN-1", header = 1)

path = "DataSources\\Income_Index.csv"
data3 = pd.read_csv(path, encoding = "LATIN-1", header = 1)

path = "DataSources\\life_expect.csv"
data4 = pd.read_csv(path, encoding = "LATIN-1", header = 1)

In [17]:
UNmaster = addUNData(data1, "HDI").join(addUNData(data2, "EDU_IDX")).join(addUNData(data3, "Income_Index")).join(addUNData(data4, "life"))
UNmaster = UNmaster.apply(pd.to_numeric)

## Now for the World Bank

In [18]:
# First there is the complicated World Bank one. This one hhas multiple indicators
path = "DataSources\\WorldBankInds.csv"
data = pd.read_csv(path)
snames = list(set(data["Series Name"].dropna()))
snames = [x for x in snames if ("Estimate" in x) or ("Standard" in x)] #We're just concerend with the errors and the estimates
snamesActual = ["corruptionSE", "ruleOfLawSE", "voice", "regQual", "govEffect", "govEffectSE", "stabilitySE", "voiceSE", 
               "corruption", "stability", "regQualSE", "ruleOfLaw"]
rename = zip(snames, snamesActual)

In [19]:
masterWB = addWorldBankVar(data[data["Series Name"] == snames[0]], snamesActual[0], True)

for cnt,name in enumerate(snames[1:]):
    datat = data[data["Series Name"] == name]
    datat = addWorldBankVar(datat, snamesActual[cnt+1], True)
    masterWB = masterWB.join(datat)
    
masterWB = masterWB.apply(pd.to_numeric)

C:\Users\Viral Shanker\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
#These are easy single indicator adds 
data = pd.DataFrame
path = "DataSources\\GDPcapitaGrowth.csv"
data = pd.read_csv(path, header = 4)

data2 = pd.DataFrame
path = "DataSources\\GDPcap.csv"
data2 = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "INCOMEPC_growth")
data2 = addWorldBankVar(data2, "INCOMEPC")
masterWB = masterWB.join(data)
masterWB = masterWB.join(data2)

data = pd.DataFrame
path = "DataSources\\trade.csv"
data = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "trade")
masterWB = masterWB.join(data)

data = pd.DataFrame
path = "DataSources\\industrial.csv"
data = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "industrial")
masterWB = masterWB.join(data)

data = pd.DataFrame
path = "DataSources\\manu.csv"
data = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "manu")
masterWB = masterWB.join(data)

data = pd.DataFrame
path = "DataSources\\agri.csv"
data = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "agri")
masterWB = masterWB.join(data)

data = pd.DataFrame
path = "DataSources\\URBAN.csv"
data = pd.read_csv(path, header = 4)
data = addWorldBankVar(data, "urban")
masterWB = masterWB.join(data)

## Finally, a table with some misc. variables

In [21]:
path = "DataSources\\financeOpen.csv"
data3 = pd.read_csv(path)
data3 = addNormalData(data3, "financeOpen")[["kaopen"]]
masterMisc = data3

In [22]:
path = "DataSources\\Polity.csv"
data3 = pd.read_csv(path)
data3["code"] = data3["country"].map(rdd)
data3 = data3[data3["code"].isin(rdd.values())]
data3 = addNormalData(data3, "polity")[["polity2"]]
masterMisc = masterMisc.join(data3)

## Finally, Finally, the last table we need to ensure normalization: code_year

In [23]:
cntr_year = []
ac = list(set(rdd.values()))
y = list(range(1996,2019))
for x in ac:
    for b in y:
        cntr_year.append([x,b])
code_year = pd.DataFrame(cntr_year, columns = ["code", "year"])

### Let's add our id to every table being in format code_year

In [24]:
code_year["id"] = code_year["code"].astype(str) + "_" + code_year["year"].astype(str)
master["g_id"] = master["code"].astype(str) + "_" + master["year"].astype(str)
UNmaster = UNmaster.assign(**UNmaster.index.to_frame())
UNmaster["un_id"] = UNmaster["code"].astype(str) + "_" + UNmaster["year"].astype(str)
masterWB = masterWB.assign(**masterWB.index.to_frame())
masterWB["wb_id"] = masterWB["code"].astype(str) + "_" + masterWB["year"].astype(str)
masterMisc = masterMisc.assign(**masterMisc.index.to_frame())
masterMisc["misc_id"] = masterMisc["code"].astype(str) + "_" + masterMisc["year"].astype(str)
master.drop(columns = ["code", "year"], inplace = True)
UNmaster.drop(columns = ["code", "year"], inplace = True)
masterWB.drop(columns = ["code", "year"], inplace = True)
masterMisc.drop(columns = ["code", "year"], inplace = True)

# Now to get these into SQL

## Create our engine SQLalchemy

In [25]:
import json

with open("init.txt") as jf:
    inp = json.load(jf)

accessString = "postgresql://" + inp["username"] + ":" + inp["password"] + "@" + inp["hostname"] + "/" + inp["database"]


from sqlalchemy import create_engine
engine = create_engine(accessString, echo = False)

In [26]:
#To completely reset our database uncomment the below line 
#engine.execute("DROP SCHEMA public CASCADE; CREATE SCHEMA public; GRANT ALL ON SCHEMA public TO postgres; GRANT ALL ON SCHEMA public TO public; COMMENT ON SCHEMA public IS 'standard public schema';")

## Simply go from pandas dataframe to table and add constraints as given in the ER diagram

In [27]:
engine.execute("DROP TABLE IF EXISTS codes_table")
codes_table.to_sql('codes_table', engine, index = False)
engine.execute("ALTER TABLE codes_table ADD PRIMARY KEY (code)")

In [28]:
engine.execute("DROP TABLE IF EXISTS country_code")
country_code.to_sql('country_code', engine, index = False)
engine.execute("ALTER TABLE country_code ADD PRIMARY KEY (cntr_name)")

In [29]:
engine.execute("ALTER TABLE country_code ADD FOREIGN KEY (code) REFERENCES codes_table (code);")

In [30]:
engine.execute("DROP TABLE IF EXISTS code_year")
code_year.to_sql('code_year', engine, index = False)
engine.execute("ALTER TABLE code_year ADD PRIMARY KEY (id)")

In [31]:
engine.execute("ALTER TABLE code_year ADD FOREIGN KEY (code) REFERENCES codes_table (code);")

In [32]:
engine.execute("DROP TABLE IF EXISTS master_gini")
master.to_sql('master_gini', engine, index = False)
engine.execute("ALTER TABLE master_gini ADD PRIMARY KEY (g_id)")

In [33]:
engine.execute("DROP TABLE IF EXISTS master_un")
UNmaster.to_sql('master_un', engine, index = False)
engine.execute("ALTER TABLE master_un ADD PRIMARY KEY (un_id)")

In [34]:
engine.execute("DROP TABLE IF EXISTS master_wb")
masterWB.to_sql('master_wb', engine, index = False)
engine.execute("ALTER TABLE master_wb ADD PRIMARY KEY (wb_id)")

In [35]:
engine.execute("DROP TABLE IF EXISTS master_misc")
masterMisc.to_sql('master_misc', engine, index = False)
engine.execute("ALTER TABLE master_misc ADD PRIMARY KEY (misc_id)")

In [36]:
engine.execute("ALTER TABLE master_gini ADD FOREIGN KEY (g_id) REFERENCES code_year (id);")

In [37]:
engine.execute("ALTER TABLE master_un ADD FOREIGN KEY (un_id) REFERENCES code_year (id);")

In [38]:
engine.execute("ALTER TABLE master_wb ADD FOREIGN KEY (wb_id) REFERENCES code_year (id);")

In [39]:
engine.execute("ALTER TABLE master_misc ADD FOREIGN KEY (misc_id) REFERENCES code_year (id);")

## ALL ENTITIES MADE!

### We have our relational databases set up! Next python notebook we will get down to the analysis